In [1]:
from falkordb import FalkorDB
import json
from sys_query import QuerySystem
import string

In [2]:
myfalkor = QuerySystem("127.0.0.1", 6379, "uni_graph")

uni_graph


In [22]:
myfalkor.update_schema_node("University", {'uni_president': "string"})

Found 1 Nodes!
(:University{founded_year:"integer",location:"string",name:"string",ranking:"integer",type:"string"})
MATCH (n:University)
                
SET n.uni_president='string' 
Node Successfully updated!


In [23]:
myfalkor.add_schema_relationship("Course", "HAS", "Research")

In [24]:
myfalkor.update_schema_relationship("Course", "Research", "MAKES")

MATCH (n1:Course)-[r]->(n2:Research)
DELETE r
CREATE (n1)-[new_r:MAKES]->(n2)


In [ ]:
myfalkor.add_node_constraints("unique", "Student", "student_id")

In [3]:
#(:Student{gpa:"float",major:"string",name:"string",status:"string",student_id:"string",year:"integer"})

myfalkor.update_kg_node("Student", {'gpa': 2.9, 'name': 'Ahmed'}, {'gpa': 3.8,  'name': "Karim", 'student_id': '1111'}, "uni_schema")

Found 1 Nodes!
(:Student{gpa:"3.8",name:"Karim",student_id:"1111"})
MATCH (n:Student)
                    WHERE n.gpa='3.8' AND n.name='Karim' AND n.student_id='1111' 
SET n.gpa='2.9' , n.name='Ahmed' 
Node Successfully updated!


In [4]:
res = myfalkor.g.query("MATCH (n:Student) RETURN (n)")
for element in res.result_set:
    print(element[0])
    #for key ,value in dict(element[0]).items():
     #   print(key, value)

(:Student{gpa:3.85,major:"Computer Science",name:"John Doe",status:"Active",student_id:"S001",year:3})
(:Student{gpa:3.92,major:"Computer Science",name:"Jane Smith",status:"Active",student_id:"S002",year:4})
(:Student{gpa:3.67,major:"Computer Science",name:"Mike Johnson",status:"Active",student_id:"S003",year:2})
(:Student{gpa:3.78,major:"Mathematics",name:"Sarah Wilson",status:"Active",student_id:"S004",year:3})
(:Student{gpa:3.55,major:"Computer Science",name:"Alex Brown",status:"Active",student_id:"S005",year:1})
(:Student{gpa:"2.9",name:"Ahmed",student_id:"1111"})


In [7]:
def load_json(path):
    with open(path, "r") as f:
        return json.load(f)
    
ontology = load_json("kg.json")

In [ ]:
for key, value in ontology["universities_data"].items():
    print(key, value["location"])

In [8]:
# Create universities
for uni, value in ontology["universities_data"].items():
    query = f"""
    CREATE (u:University {{
        name: '{value["name"]}',
        location: '{value["location"]}',
        founded_year: {value["founded_year"]},
        type: '{value["type"]}',
        ranking: {value["ranking"]}
    }})
    """
    myfalkor.g.query(query)

# Create departments
for dept, value in ontology["departments_data"].items():
    query = f"""
    MATCH (u:University {{name: '{value["university"]}'}})
    CREATE (d:Department {{
        dept_name: '{value["name"]}',
        budget: {value["budget"]},
        head: '{value["head"]}',
        student_count: {value["student_count"]},
        building: '{value["name"]} Building'
    }})
    CREATE (d)-[:PART_OF]->(u)
    """
    myfalkor.g.query(query)

# Create professors
for prof, val in ontology["professors_data"].items():
    query = f"""
    MATCH (d:Department {{dept_name: '{val["department"]}'}})-[:PART_OF]->(u:University {{name: '{val["university"]}'}})
    CREATE (p:Professor {{
        name: '{val["name"]}',
        title: '{val["title"]}',
        research_area: '{val["research_area"]}',
        tenure: '{val["tenure"]}',
        email: '{val["email"]}',
        office: '{val["office"]}'
    }})
    CREATE (p)-[:WORKS_IN]->(d)
    """
    myfalkor.g.query(query)

# Create students
for student, val in ontology["students_data"].items():
    query = f"""
    MATCH (d:Department {{dept_name: '{val["major"]}'}})-[:PART_OF]->(u:University {{name: '{val["university"]}'}})
    CREATE (s:Student {{
        name: '{val["name"]}',
        student_id: '{val["student_id"]}',
        major: '{val["major"]}',
        gpa: {val["gpa"]},
        year: {val["year"]},
        status: '{val["status"]}'
    }})
    CREATE (s)-[:BELONGS_TO]->(d)
    """
    myfalkor.g.query(query)

# Create courses
for course, val in ontology["courses_data"].items():
    dept_name = "Computer Science" if val["code"].startswith(('CS', '6.')) else "Mathematics"
    query = f"""
    MATCH (d:Department {{dept_name: '{dept_name}'}})-[:PART_OF]->(u:University {{name: '{val["university"]}'}})
    CREATE (c:Course {{
        code: '{val["code"]}',
        name: '{val["name"]}',
        credits: {val["credits"]},
        level: '{val["level"]}',
        semester: '{val["semester"]}',
        capacity: {val["capacity"]}
    }})
    CREATE (c)-[:OFFERED_BY]->(d)
    """
    myfalkor.g.query(query)

# Create buildings
for building, val in ontology["buildings_data"].items():
    query = f"""
    MATCH (u:University {{name: '{val["university"]}'}})
    CREATE (b:Building {{
        name: '{val["name"]}',
        address: '{val["address"]}',
        floors: {val["floors"]},
        capacity: {val["capacity"]},
        type: '{val["type"]}'
    }})
    CREATE (b)-[:LOCATED_AT]->(u)
    """
    myfalkor.g.query(query)



# Create teaching relationships
teaching_relationships = {
("Dr. Alice Johnson", "CS229"),
("Dr. Bob Smith", "CS231N"),
("Dr. Carol Davis", "CS106A"),
("Dr. David Wilson", "6.034"),
("Dr. Eva Martinez", "18.06")
}

for prof_name, course_code in teaching_relationships:
    query = f"""
    MATCH (p:Professor {{name: '{prof_name}'}}),
            (c:Course {{code: '{course_code}'}})
    CREATE (p)-[:TEACHES]->(c)
    """
    myfalkor.g.query(query)

# Create enrollment relationships
enrollments = [
    ("John Doe", "CS106A"),
    ("John Doe", "CS229"),
    ("Jane Smith", "CS229"),
    ("Jane Smith", "CS231N"),
    ("Mike Johnson", "6.034"),
    ("Sarah Wilson", "18.06"),
    ("Alex Brown", "CS106A")
]

for student_name, course_code in enrollments:
    query = f"""
    MATCH (s:Student {{name: '{student_name}'}}),
            (c:Course {{code: '{course_code}'}})
    CREATE (s)-[:ENROLLED_IN]->(c)
    """
    myfalkor.g.query(query)

# Link courses to buildings
course_building_links = [
    ("CS106A", "Gates Computer Science Building"),
    ("CS229", "Gates Computer Science Building"),
    ("CS231N", "Gates Computer Science Building"),
    ("6.034", "Stata Center"),
    ("18.06", "Stata Center")
]

for course_code, building_name in course_building_links:
            query = f"""
            MATCH (c:Course {{code: '{course_code}'}}),
                  (b:Building {{name: '{building_name}'}})
            CREATE (c)-[:HELD_IN]->(b)
            """
            myfalkor.g.query(query)

In [9]:
myfalkor.db.list_graphs()

['uni_schema', 'uni_graph']

In [10]:
res = myfalkor.g.query("MATCH (n:Student) RETURN properties(n)")
for element in res.result_set:
    print(dict(element[0]))
    #for key ,value in dict(element[0]).items():
     #   print(key, value)

{'name': 'John Doe', 'student_id': 'S001', 'major': 'Computer Science', 'gpa': 3.85, 'year': 3, 'status': 'Active'}
{'name': 'Jane Smith', 'student_id': 'S002', 'major': 'Computer Science', 'gpa': 3.92, 'year': 4, 'status': 'Active'}
{'name': 'Mike Johnson', 'student_id': 'S003', 'major': 'Computer Science', 'gpa': 3.67, 'year': 2, 'status': 'Active'}
{'name': 'Sarah Wilson', 'student_id': 'S004', 'major': 'Mathematics', 'gpa': 3.78, 'year': 3, 'status': 'Active'}
{'name': 'Alex Brown', 'student_id': 'S005', 'major': 'Computer Science', 'gpa': 3.55, 'year': 1, 'status': 'Active'}


In [4]:
myfalkor.create_schema("schema.json")

No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema
No Nodes Found!
Successfully added 1.0 node(s) to the schema


In [3]:
myfalkor.db.flushdb()

True

In [26]:
myfalkor.add_schema_node("BusinessMan", {'name': 'string', 'age': 'integer'})

name: 'string', age: 'integer'
CREATE (n:BusinessMan 
         {name: 'string', age: 'integer'})
1.0


In [28]:
myfalkor.add_schema_relationship("BusinessMan", "OWN", "University")

In [6]:
res = myfalkor.g.query("MATCH (n) RETURN (n) AS nodes")
for element in res.result_set:
    print(element[0])

(:University{founded_year:"integer",location:"string",name:"string",ranking:"integer",type:"string"})
(:Department{budget:"float",building:"string",dept_name:"string",head:"string",student_count:"integer"})
(:Professor{email:"string",name:"string",office:"string",research_area:"string",tenure:"boolean",title:"string"})
(:Student{gpa:"float",major:"string",name:"string",status:"string",student_id:"string",year:"integer"})
(:Course{capacity:"integer",code:"string",credits:"integer",level:"string",name:"string",semester:"string"})
(:Building{address:"string",capacity:"integer",floors:"integer",name:"string",type:"string"})
(:Research{duration:"integer",field:"string",funding:"float",status:"string",title:"string"})
(:Publication{citations:"integer",journal:"string",title:"string",type:"string",year:"integer"})
(:Degree{duration:"integer",level:"string",name:"string",requirements:"string"})
(:Semester{end_date:"string",name:"string",start_date:"string",year:"integer"})


In [6]:
res = myfalkor.g.ro_query("call db.constraints()")

for row in res.result_set:
    print(row)


['MANDATORY', 'Student', ['name'], 'NODE', 'OPERATIONAL']
['UNIQUE', 'Student', ['student_id'], 'NODE', 'OPERATIONAL']


In [ ]:
myfalkor.g.query("CREATE INDEX FOR (b:BusinessMan) ON (b.id)")

In [5]:
myfalkor.add_node_constraints("mandatory", "Student", "name")

Found 1 Nodes!
(:Student{gpa:"float",major:"string",name:"string",status:"string",student_id:"string",year:"integer"})


In [30]:
myfalkor.update_schema_relationship(node1='BusinessMan', node2='University', new_rel='CONTROL', new_rel_attr={'date': 'date'})

MATCH (n1:BusinessMan)-[r]->(n2:University)
DELETE r
CREATE (n1)-[new_r:CONTROL]->(n2)
SET new_r.date= 'date'


In [31]:
res =myfalkor.g.query("MATCH (n)-[r]->(m) RETURN labels(n)[0] AS NODES, labels(m)[0] AS NODES2, type(r) AS RELATIONSHIP")
print(res.result_set)
for row in res.result_set:
    print(f"{row[0]} -- [{row[2]}] --> {row[1]}")

[['Department', 'University', 'PART_OF'], ['Professor', 'Department', 'WORKS_IN'], ['Professor', 'Course', 'TEACHES'], ['Professor', 'Research', 'CONDUCTS'], ['Professor', 'Publication', 'AUTHORED'], ['Student', 'Department', 'BELONGS_TO'], ['Student', 'Course', 'ENROLLED_IN'], ['Student', 'Degree', 'PURSUING'], ['Course', 'Department', 'OFFERED_BY'], ['Course', 'Building', 'HELD_IN'], ['Course', 'Semester', 'OFFERED_IN'], ['Building', 'University', 'LOCATED_AT'], ['Research', 'University', 'FUNDED_BY'], ['BusinessMan', 'University', 'CONTROL']]
Department -- [PART_OF] --> University
Professor -- [WORKS_IN] --> Department
Professor -- [TEACHES] --> Course
Professor -- [CONDUCTS] --> Research
Professor -- [AUTHORED] --> Publication
Student -- [BELONGS_TO] --> Department
Student -- [ENROLLED_IN] --> Course
Student -- [PURSUING] --> Degree
Course -- [OFFERED_BY] --> Department
Course -- [HELD_IN] --> Building
Course -- [OFFERED_IN] --> Semester
Building -- [LOCATED_AT] --> University
Rese

In [6]:
myfalkor.find_node("Exams", {'id': "string", 'date': 'date'}, make_exist=True, check_schema='uni_schema')

MATCH (n:Exams)
WHERE n.id='string' AND n.date='date' 
RETURN n
No Nodes Found!
id: 'string', date: 'date'
CREATE (n:Exams 
         {id: 'string', date: 'date'})
1.0


In [14]:
res = myfalkor.g.query("MATCH (n:Student) WHERE n.id='114' RETURN (n)")
for element in res.result_set:
    print(element[0])